In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

## We have two dataframes consisting data of same movies so we will merge them and have common data frame

In [ ]:
movies = movies.merge(credits,on="title")

In [ ]:
### Feature engineerin
## content based recommendation ?

# Ask yourself wether it help to make tag 

In [ ]:
#genre, keywords, title , overview, cast, crew

# these thigs will add

In [ ]:
movies = movies[['movie_id', 'title', 'overview','genres','keywords','cast','crew']] 

# I have to make tags  , overview , geners , keywords, top 3 cast, top 3 crew

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres
# List of distion se seedha list mein laana 

# This is string of list ko convert list mein krna hoga 




In [ ]:

def convertDicttoList(obj):
    import ast
    
    genreList=[]

    # this convert tje strinf into lsit
    for i in ast.literal_eval(obj):
        genreList.append(i['name'])
        
    # print(genereList)
    return genreList

s = convertDicttoList(movies.iloc[0].genres)

print(s)

In [ ]:
print(movies.iloc[0].genres)

In [ ]:
# Here we are applying this to every row genere in movies section and storing it in the same place

movies['genres']=movies['genres'].apply(convertDicttoList)

### Same thing with keyword column

In [ ]:
movies['keywords'] = movies['keywords'].apply(convertDicttoList)

#### Cast and Crew now  isme se mujhe first three he chahiye



In [ ]:

def convertCrewList(obj):
    import ast
    genreList=[]

    counter = 0
    # this convert tje strinf into lsit
    for i in ast.literal_eval(obj):
        if counter>2:
            return genreList
            
        genreList.append(i['name'])
        counter = counter +1

    # print(genereList)
    return genreList

In [ ]:
def fetch_director(obj):
    import ast
    l = []

    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l


# movies['crew'] =
# movies['crew'] = movies['crew'].apply(fetch_director)

movies['cast'] = movies['cast'].apply(convertCrewList)

In [ ]:
# Overview is syting to convert into list 

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

### Now removing the spaces as space contain different entity as Sam Washington will get two enityt  hence the model will get confused to someone with sam Singh to avoid this confusion we are removing the space

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
# saying space ki jagah hata dp

movies['cast']

In [ ]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [ ]:
movies.head()

<!--  -->

In [ ]:
# Creating  A tag column here 

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['crew'] + movies['cast']

# list of words 
movies['tags']

#### Now creating new dataframe so that only tags and movie name and id should be there

In [ ]:
df = movies[["movie_id", 'title', 'tags']]

df["tags"]

In [ ]:
!pip install nltk

In [ ]:
# converting list to string

df["tags"] = df["tags"].apply(lambda x: " ".join(x))


### Vectorisation of the data


In [ ]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()




In [ ]:
def stem(text):
    y =[]

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
df['tags'] = df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


cv = CountVectorizer(max_features = 5000, stop_words='english')

In [ ]:
# By default count vecotriser ek sparce matrix hoti hai

vectors = cv.fit_transform(df["tags"]).toarray()

In [ ]:
# Calculating distance between each vector with each vector 
# We will calculate cosine simmilarity 
# Euclidiean distance is not useful in higher dimension 
# Hence we use cosine similaraity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


similarity  = cosine_similarity(vectors)